<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/Chronos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install pmdarima

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-rd_anlpf
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-rd_anlpf
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 1f099eb265a4b423529929321929d4258dc031d8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 53.2 MB/s eta 0:00:00
  Created wheel for chronos-forecasting: filename=chronos_forecasting-2.2.2-py3-none-any.whl size=73518 sha256=bbe891eff8b6bbb4a846ce1fdd87ef098867ff024e84167501abac87714a50f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-rd5u7jye/wheels/b9/a6/b5/75fca7306751a3bc92a63680f861f44a42a8776f6423cf0188
Successfully built chronos-forecasting
  Att

In [2]:
import pandas as pd
import numpy as np
import torch
import warnings
from chronos import ChronosPipeline

warnings.filterwarnings("ignore")

# CARREGAMENTO DOS DADOS
df = pd.read_excel('data.xlsx')
df['Data'] = pd.to_datetime(df['Data'])
df = df.sort_values('Data').set_index('Data')

# Configurações
colunas_alvo = ['INCC- DI_var_mes', 'Taxa_SELIC_avg', 'CUB Global', 'Cub Material',
                'CUB Mão de Obra', 'CUB Desp. Administrativa', 'CUB Equipamento']
contexts = [24, 60, 120]
horizons = [12, 24]
n_windows = 10
NUM_SAMPLES = 20 # Amostragem para cálculo do CRPS probabilístico

# FUNÇÕES DE MÉTRICAS
def calculate_mase(y_true, y_pred, y_train, seasonal_period=12):
    denom = np.mean(np.abs(np.diff(y_train, seasonal_period)))
    if denom == 0: return np.nan
    return np.mean(np.abs(y_true - y_pred)) / denom

def calculate_crps_samples(y_true, samples):
    """Calcula o CRPS empírico via amostras (Monte Carlo)"""
    y_true = np.array(y_true)
    term1 = np.mean(np.abs(samples - y_true), axis=0)
    s_i = samples[:, np.newaxis, :]
    s_j = samples[np.newaxis, :, :]
    term2 = np.mean(np.abs(s_i - s_j), axis=(0, 1))
    return np.mean(term1 - 0.5 * term2)

# INICIALIZAÇÃO DO CHRONOS
print("Inicializando Chronos-T5-Base na GPU...")
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

results_chronos = []

# LOOP DE EXPERIMENTAÇÃO (ROLLING ORIGIN)
for col in colunas_alvo:
    series = df[col].dropna()
    print(f"Executando Chronos: {col}")

    for C in contexts:
        for H in horizons:
            mase_list, crps_list = [], []

            for w in range(n_windows):
                end_idx = len(series) - w
                start_idx = end_idx - (C + H)
                if start_idx < 0: break

                train = series.iloc[start_idx:start_idx+C].values
                test = series.iloc[start_idx+C:end_idx].values

                try:
                    # O Chronos gera a distribuição futura baseada no contexto C
                    context_tensor = torch.tensor(train)
                    forecast_samples = pipeline.predict(
                        context_tensor,
                        prediction_length=H,
                        num_samples=NUM_SAMPLES,
                    ).numpy()[0]

                    # Mediana das amostras para métrica de ponto
                    forecast_median = np.median(forecast_samples, axis=0)

                    mase_list.append(calculate_mase(test, forecast_median, train))
                    crps_list.append(calculate_crps_samples(test, forecast_samples))
                except Exception as e:
                    continue

            if mase_list:
                results_chronos.append({
                    'Variável': col, 'C': C, 'H': H,
                    'MASE (Mediana)': round(np.median(mase_list), 4),
                    'CRPS (Mediana)': round(np.median(crps_list), 4),
                    'Modelo': 'Chronos-Base'
                })

# RESULTADOS FINAIS
results_df_chronos = pd.DataFrame(results_chronos)
print("\n" + "="*40)
print("RESULTADOS FINAIS - CHRONOS")
print("="*40)
print(results_df_chronos.to_string(index=False))

# Salvar resultados em csv
results_df_chronos.to_csv('RESULTADOS_Chronos.csv', index=False)

Inicializando Chronos-T5-Base na GPU...


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Executando Chronos: INCC- DI_var_mes
Executando Chronos: Taxa_SELIC_avg
Executando Chronos: CUB Global
Executando Chronos: Cub Material
Executando Chronos: CUB Mão de Obra
Executando Chronos: CUB Desp. Administrativa
Executando Chronos: CUB Equipamento

RESULTADOS FINAIS - CHRONOS
                Variável   C  H  MASE (Mediana)  CRPS (Mediana)       Modelo
        INCC- DI_var_mes  24 12          0.0008          0.2343 Chronos-Base
        INCC- DI_var_mes  24 24          0.0008          0.2214 Chronos-Base
        INCC- DI_var_mes  60 12          0.0009          0.1886 Chronos-Base
        INCC- DI_var_mes  60 24          0.0010          0.1781 Chronos-Base
        INCC- DI_var_mes 120 12          0.0007          0.1834 Chronos-Base
        INCC- DI_var_mes 120 24          0.0006          0.1722 Chronos-Base
          Taxa_SELIC_avg  24 12          0.0165          0.9082 Chronos-Base
          Taxa_SELIC_avg  24 24          0.0185          1.7305 Chronos-Base
          Taxa_SELIC_avg 